# TensorFlow Estimators

```{admonition} Attribution
This notebook builds on Chapter 14: *Going Deeper – The Mechanics of TensorFlow* of {cite}`RaschkaMirjalili2019`.
```

The `tf.estimator` API
encapsulates the underlying steps in machine learning tasks, such as training,
inference, and evaluation. Estimators are more encapsulated but also
more scalable when compared to the previous approaches that we have covered
in the previous notebook. Also, the `tf.estimator` API adds support for running models on
multiple platforms without requiring major code changes, which makes them more
suitable for the so-called "production phase" in industry applications. In addition,
TensorFlow comes with a selection of off-the-shelf estimators for common machine
learning and deep learning architectures that are useful for comparison studies, for
example, to quickly assess whether a certain approach is applicable to a particular
dataset or problem.

## Working with feature columns

Note that while numeric data can be either continuous or discrete,
in the context of the TensorFlow API, "numeric" data specifically refers to continuous
data of the floating point type.